In [1]:
from pathlib import Path
import time
import pandas as pd
import geopandas as gpd
import numpy as np
import pickle
from stochopy.optimize import minimize
from tqdm import tqdm
import similaritymeasures
import random
import matplotlib.pyplot as plt
from shapely.ops import Point, MultiLineString, LineString
from importlib import reload
import datetime
from scipy.spatial.distance import directed_hausdorff
np.set_printoptions(suppress=True)
from matplotlib.ticker import MultipleLocator

from bikewaysim.paths import config, stadia_toner, maptiler_streets
from bikewaysim.impedance_calibration import stochastic_optimization, speedfactor
from bikewaysim.network import modeling_turns
from bikewaysim.routing import rustworkx_routing_funcs
from bikewaysim.impedance_calibration import optimization_viz, utils, post_calibration, impedance_functions, loss_functions

In [2]:
with (config['calibration_fp'] / 'results/bootstrap_final,validation,0.pkl').open('rb') as fh:
    bootstrap_model = pickle.load(fh)

links, turns, length_dict, geo_dict, turn_G = rustworkx_routing_funcs.import_calibration_network(config)

In [3]:
from importlib import reload
reload(stochastic_optimization)
 
# links['link_cost']

# run impedance routing on all the provided ODs
base_impedance_col = "travel_time_min"
rustworkx_routing_funcs.back_to_base_impedance(base_impedance_col,links,turns,turn_G)

betas_tup = bootstrap_model['betas_tup']
betas = [x['beta'] for x in betas_tup] # get betas

#update impedances
print(betas)
_ = rustworkx_routing_funcs.impedance_update(
    betas,betas_tup,
    impedance_functions.link_impedance_function,
    base_impedance_col,
    None,
    impedance_functions.turn_impedance_function,
    links,turns,turn_G)

[0.1225, 0.8465, 0.371, 0.873, 1.815, -0.493, -0.5115000000000001, -0.3655, 1.1675, 0.706, 0.7215]


In [4]:
highest_multiplier = links.groupby('linkid')['multiplier'].apply(lambda x: x.abs().idxmax()).tolist()
highest_multiplier = links.loc[highest_multiplier]
highest_multiplier['pct'] = highest_multiplier['multiplier'] * 100
highest_multiplier[['travel_time_min','link_cost','multiplier','pct']]

,travel_time_min,link_cost,multiplier,pct
49248,1.115741,2.474155,1.2175,121.75
49085,0.616576,0.845325,0.3710,37.10
49087,0.939707,1.288338,0.3710,37.10
49141,0.703865,0.964999,0.3710,37.10
49187,0.654517,0.897342,0.3710,37.10
...,...,...,...,...
345869,0.076105,0.142544,0.8730,87.30
345870,0.121866,0.343053,1.8150,181.50
345845,0.396170,0.396170,0.0000,0.00
345841,0.294664,0.294664,0.0000,0.00


In [5]:
highest_multiplier.loc[highest_multiplier['multiplier'] > 0,['travel_time_min','link_cost','multiplier','pct','geometry']].to_file(config['calibration_fp']/"network_impedance_change2.gpkg",layer='increased_impedance')
highest_multiplier.loc[highest_multiplier['multiplier'] < 0,['travel_time_min','link_cost','multiplier','pct','geometry']].to_file(config['calibration_fp']/"network_impedance_change2.gpkg",layer='decreased_impedance')
highest_multiplier.loc[highest_multiplier['multiplier'] == 0,['travel_time_min','link_cost','multiplier','pct','geometry']].to_file(config['calibration_fp']/"network_impedance_change2.gpkg",layer='base_case')

## For the table 

In [12]:
beta_vec

[0.8465, 0.371, 0.873, 1.815, -0.493, -0.3655]

In [13]:
cols

['3+lpd',
 '(30,inf) mph',
 '[4,6) grade',
 '[6,inf) grade',
 'bike lane',
 'multi use path']

In [6]:
cols = ['3+lpd','(30,inf) mph','[4,6) grade','[6,inf) grade','bike lane','multi use path']
beta_vec = [x['beta'] for x in betas_tup if x['col'] in cols]
beta_vec

[0.8465, 0.371, 0.873, 1.815, -0.493, -0.3655]

In [15]:
test = {key:val for key, val in zip(cols,beta_vec)}
test

{'3+lpd': 0.8465,
 '(30,inf) mph': 0.371,
 '[4,6) grade': 0.873,
 '[6,inf) grade': 1.815,
 'bike lane': -0.493,
 'multi use path': -0.3655}

In [ ]:
Grade (\%) 	& Speed (MPH) & Lanes Per Direction & Pedestrian Path & Multi-Use Path & Road & Road with Bike Lane

In [ ]:
full_table = np.array([
    []
]


[0.8465, 0.371, 0.873, 1.815, -0.493, -0.3655]

In [7]:
x = np.array([
    [0,0,0,0,0,0],
    [0,0,1,0,0,0],
    [0,0,0,1,0,0],
    [0,0,0,0,0,1],
    [0,0,1,0,0,1],
    [0,0,0,1,0,1],
    [0,0,0,0,0,0],
    [0,0,1,0,0,0],
    [0,0,0,1,0,0],
    [1,0,0,0,0,0],
    [1,0,1,0,0,0],
    [1,0,0,1,0,0],
    [1,1,0,0,0,0],
    [1,1,1,0,0,0],
    [1,1,0,1,0,0],
    [0,1,0,0,0,0],
    [0,1,1,0,0,0],
    [0,1,0,1,0,0],
    [0,0,0,0,1,0],
    [0,0,1,0,1,0],
    [0,0,0,1,1,0],
    [1,0,0,0,1,0],
    [1,0,1,0,1,0],
    [1,0,0,1,1,0],
    [1,1,0,0,1,0],
    [1,1,1,0,1,0],
    [1,1,0,1,1,0],
    [0,1,0,0,1,0],
    [0,1,1,0,1,0],
    [0,1,0,1,1,0]
])

In [8]:
stats = np.sum(x * beta_vec,axis=1).round(2)
stats

array([ 0.  ,  0.87,  1.82, -0.37,  0.51,  1.45,  0.  ,  0.87,  1.82,
        0.85,  1.72,  2.66,  1.22,  2.09,  3.03,  0.37,  1.24,  2.19,
       -0.49,  0.38,  1.32,  0.35,  1.23,  2.17,  0.72,  1.6 ,  2.54,
       -0.12,  0.75,  1.69])

## Basic Stats

In [9]:
stats

array([ 0.  ,  0.87,  1.82, -0.37,  0.51,  1.45,  0.  ,  0.87,  1.82,
        0.85,  1.72,  2.66,  1.22,  2.09,  3.03,  0.37,  1.24,  2.19,
       -0.49,  0.38,  1.32,  0.35,  1.23,  2.17,  0.72,  1.6 ,  2.54,
       -0.12,  0.75,  1.69])

In [10]:
print(stats.max())
print(stats.min())

3.03
-0.49
